In [1]:
#packages
import scipy.io
import math
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
#loading data
data = scipy.io.loadmat("fashion_mnist.mat") 

NAIVE BAYES

In [3]:
#loading train data
xtraind = data['trX']
ytraind = data['trY'] 

In [4]:
#computing mean and standard deviation
xtraind_df = pd.DataFrame(xtraind)
xdata = {'mean': xtraind_df.mean(axis=1), 'std' : xtraind_df.std(axis=1)}
xdata_f = pd.DataFrame(xdata)

In [5]:
ytraind_df = pd.DataFrame(ytraind)

In [6]:
xdata_f['label'] = pd.DataFrame(ytraind_df.T)

In [7]:
dataclass_0 = xdata_f.loc[xdata_f['label']== 0]
dataclass_1 = xdata_f.loc[xdata_f['label']== 1]

In [8]:
means_class0 = [0,0]
means_class1 = [0,0]
vars_class0 = [0,0]
vars_class1 = [0,0]

In [9]:
#computing means for class 0 and 1
means_class0[0] = dataclass_0['mean'].mean()
means_class0[1] = dataclass_0['std'].mean()
means_class1[0] = dataclass_1['mean'].mean()
means_class1[1] = dataclass_1['std'].mean()

In [10]:
#computing variance for class 0 and 1
vars_class0[0] = dataclass_0['mean'].var()
vars_class0[1] = dataclass_0['std'].var()
vars_class1[0] = dataclass_1['mean'].var()
vars_class1[1] = dataclass_1['std'].var()

In [11]:
#initialising prior probabilities
prior_prob0 = 0.5
prior_prob1 = 0.5

In [12]:
#defining pdf
def pdf(x , mean,var):
    e = -0.5*((x - mean)** 2 / var)
    return (1.0 / (math.sqrt(2*math.pi*var)) * math.exp(e))

In [13]:
#calculating prediction data frame and class probabilities
pred_df = pd.DataFrame(columns = ['Prediction_label'])
for i in range(xdata_f.shape[0]):
    prob_class0 = pdf(xdata_f['mean'][i],means_class0[0],vars_class0[0])*pdf(xdata_f['std'][i],means_class0[1],vars_class0[1])*prior_prob0
    prob_class1 = pdf(xdata_f['mean'][i],means_class1[0],vars_class1[0])*pdf(xdata_f['std'][i],means_class1[1],vars_class1[1])*prior_prob1
    
    if prob_class0 > prob_class1:
        pred_df.loc[i, 'Prediction_label'] = 0.0
    else:
        pred_df.loc[i, 'Prediction_label'] = 1.0

In [15]:
pred_df['Prediction_label']= pred_df.Prediction_label.astype(float)

In [16]:
#accuracy score for train data
accuracy_score(xdata_f['label'], pred_df['Prediction_label'])*100

82.38333333333333

In [17]:
confusion_matrix(xdata_f['label'], pred_df['Prediction_label'])


array([[4520, 1480],
       [ 634, 5366]])

In [18]:
#proof
(5366+4520)/(1480+634+5366+4520)

0.8238333333333333

In [19]:
#loading test data
X_test = data['tsX']
Y_test = data['tsY']

In [20]:
x_test_df = pd.DataFrame(X_test)

In [21]:
#computing mean and standard deviation
xdata2 = {'mean': x_test_df.mean(axis=1), 'std' : x_test_df.std(axis=1)}
x_testdata_f= pd.DataFrame(xdata2)

In [22]:
Y_testdata_df = pd.DataFrame(Y_test)
x_testdata_f['label'] = pd.DataFrame(Y_testdata_df.T)

In [23]:
#calculating test data prediction frame and class probabilities
test_pred_df = pd.DataFrame(columns = ['Prediction_label'])
for i in range(x_testdata_f.shape[0]):
    prob_class0 = pdf(x_testdata_f['mean'][i],means_class0[0],vars_class0[0])*pdf(x_testdata_f['std'][i],means_class0[1],vars_class0[1])*prior_prob0
    prob_class1 = pdf(x_testdata_f['mean'][i],means_class1[0],vars_class1[0])*pdf(x_testdata_f['std'][i],means_class1[1],vars_class1[1])*prior_prob1
    
    if prob_class0 > prob_class1:
        test_pred_df.loc[i, 'Prediction_label'] = 0.0
    else:
        test_pred_df.loc[i, 'Prediction_label'] = 1.0

In [24]:
test_pred_df['Prediction_label']= test_pred_df.Prediction_label.astype(float)


In [25]:
#accuracy% for test data
accuracy_score(x_testdata_f['label'], test_pred_df['Prediction_label'])*100

83.15

In [26]:
confusion_matrix(x_testdata_f['label'], test_pred_df['Prediction_label'])

array([[784, 216],
       [121, 879]])

LOGISTIC REGRESSION

In [27]:
#initialising weights, bias
#lr = learning rate, no_itr=number of iterations
def train_data( x, y, lr, no_itr ):
    weights = np.zeros(x.shape[1])
    bias = 0
    #computing sigmoid
    for i in range(no_itr):
      y_pred = np.dot(weights,x.T) + bias
      y_pred = 1/(1 + np.exp(-y_pred))

      prediction_error = y-y_pred
      d = np.average(prediction_error*x.T,1)

      #weights update
      weights = weights + d*lr

      #computing cost function
      log_likelihood = np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

      #updating bias
      bias  = bias + np.average(prediction_error)
    return weights,bias 


In [28]:
#prediction function
def prediction(xv, wt, bias):
  y_pred = np.dot(wt,xv.T) + bias
  y_pred[np.where(y_pred>=0.5)]=1
  y_pred[np.where(y_pred<0.5)]=0
  return y_pred

In [29]:
#calculating accuracy
def accuracy_calc(y_pred, y_given):
  x = y_pred == y_given
  acc = (np.sum(x)/y_pred.shape[0]) * 100    
  return acc

In [30]:
#train data
x_train_w = xdata_f[['mean','std']]
x_test_w = x_testdata_f[['mean','std']]

In [ ]:
#calculating accuracy
weights,bias = train_data(x_train_w.to_numpy(), ytraind_df.to_numpy(), 1.5, 9000)
prediction_classes = prediction(x_test_w.to_numpy(),weights,bias)
print(accuracy_calc(prediction_classes,Y_testdata_df.to_numpy() ))